In [ ]:
#@title Copyright 2024 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
%%file requirements.txt
google-auth

In [ ]:
%%capture
!pip install -r requirements.txt

In [ ]:
%%writefile main.py
from markupsafe import escape
import google.auth

credentials, project_id = google.auth.default();

def hello_http(request):
    """HTTP Cloud Function.
    Args:
        request (flask.Request): The request object.
        <https://flask.palletsprojects.com/en/1.1.x/api/#incoming-request-data>
    Returns:
        The response text, or any set of values that can be turned into a
        Response object using `make_response`
        <https://flask.palletsprojects.com/en/1.1.x/api/#flask.make_response>.
    """
    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and "name" in request_json:
        name = request_json["name"]
    elif request_args and "name" in request_args:
        name = request_args["name"]
    else:
        name = "World"
    return f"Heya {escape(name)}, I'm a function running on projects/{project_id or 'unknown'}!"

In [ ]:
import json
import importlib
import main
importlib.reload(main) # Force reload of the main file.
from flask import Request

from google.colab import auth
PROJECT_ID = "mdewitt-earthengine" # @param {"type":"string"}
auth.authenticate_user(project_id=PROJECT_ID)

data = {
    "name": "Michael",
}

request = Request.from_values(
    headers={'Content-Type': 'application/json'},
    data=json.dumps(data),
    method="POST")

print(main.hello_http(request))

In [ ]:
!gcloud functions deploy hello_http --runtime python312 --trigger-http\
  --allow-unauthenticated\
  --project $PROJECT_ID